In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：543



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}",5756,8304960,MELLING;30250S;5756;8304960
1,2,Engine,Timing Chain & Component Kit,MELLING,30250SX,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""Does Not Include Sprockets""}","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$135.79""}",5756,11158045,MELLING;30250SX;5756;11158045
2,3,Engine,Timing Chain & Component Kit,MELLING,30420S,,,"https://www.rockauto.com/en/parts/melling,3042...","{""0"": ""https://www.rockauto.com/info/583/3-042...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$139.79""}",5756,8304968,MELLING;30420S;5756;8304968
3,4,Engine,Timing Chain & Component Kit,MELLING,30430S,,,"https://www.rockauto.com/en/parts/melling,3043...","{""0"": ""https://www.rockauto.com/info/583/3-043...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$188.79""}",5756,8304972,MELLING;30430S;5756;8304972
4,5,Engine,Timing Chain & Component Kit,MELLING,30460S,,,"https://www.rockauto.com/en/parts/melling,3046...","{""0"": ""https://www.rockauto.com/info/583/3-046...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""Does Not Include Sprockets""}","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$223.79""}",5756,8304976,MELLING;30460S;5756;8304976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,539,Engine,Timing Chain & Component Kit,MELLING,TOPK1014,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes Standard Volume/Pressure Oil P...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$1,106.79""}",5756,15383849,MELLING;TOPK1014;5756;15383849
539,540,Engine,Timing Chain & Component Kit,MELLING,TOPK1015,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes Oil Pump, Variable Valve Timin...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$303.79""}",5756,15583693,MELLING;TOPK1015;5756;15583693
540,541,Engine,Timing Chain & Component Kit,MELLING,TOPK1015HV,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes High Volume Oil Pump, Variable...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$333.79""}",5756,15583697,MELLING;TOPK1015HV;5756;15583697
541,542,Engine,Timing Chain & Component Kit,MELLING,TOPK1016,,,"https://www.rockauto.com/en/parts/melling,TOPK...",{},https://www.rockauto.com/en/moreinfo.php?pk=16...,"{""0"": ""Includes Oil Pump, Variable Valve Timin...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$429.79""}",5756,16272549,MELLING;TOPK1016;5756;16272549


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 OEM                                                              
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Json_Src        {"0": "https://www.rockauto.com/info/583/3-025...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                                                    {}
 Json_Note 2       {"0": "Category: Timing Chain & Component Kit"}
 Choose                                                           
 Json_Price                                       {"0": "$171.79"}
 Type Code                                        

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['Info']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() == 'Continue Shopping':
                with open('./info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

soup

1


<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
<head>
<meta content="all" name="robots"/>
<meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
<link href="/jslib/combined/css/desktopmoreinfo.css?1729207456" rel="stylesheet" type="text/css"/>
<script>
if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for(var d=windo

In [6]:
list_tr, dict_count = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else [], {}
for i, tr in enumerate(list_tr):
    list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if len(list_text) > 2:
        print('fuck')
    elif list_text[0] in dict_count:
        dict_count[list_text[0]] += 1
    else:
        dict_count[list_text[0]] = 1

dict_specification, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    dict_specification[str(i)] = {}

    list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if dict_count[list_text[0]] == 1:
        for j, text in enumerate(list_text):
            dict_specification[str(i)][str(j)] = text
    else:
        if list_text[0] in dict_no:
            dict_no[list_text[0]] += 1
        else:
            dict_no[list_text[0]] = 1
        for j, text in enumerate(list_text):
            if j == 0:
                dict_specification[str(i)][str(j)] = f'{text}-{dict_no[list_text[0]]}'
            else:
                dict_specification[str(i)][str(j)] = text

dict_specification

{'0': {'0': 'Camshaft Sprocket Material', '1': 'High Alloy Steel'},
 '1': {'0': 'Keyway Adjustable', '1': 'No'},
 '2': {'0': 'Sprockets Included', '1': 'Yes'},
 '3': {'0': 'Timing Chain Type', '1': 'Silent'}}

In [7]:
dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in enumerate(list_tr):
    dict_kit[str(i)] = {}

    list_td = tr.xpath('./td')
    dict_kit[str(i)]['[Kit] Quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] Type'] = list_td[1].xpath('./text()')[0].strip()

    # = = = = = = = = = = = = = = =

    dict_ = json.loads(tr.xpath('./td[contains(@name, "listing_data_essential[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Type Code'] = dict_['parttype']
    dict_kit[str(i)]['[Kit] Part Code'] = dict_['partkey']

    # = = = = = = = = = = = = = = =

    dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Part Number'] = dict_2['partnumber']
    dict_kit[str(i)]['[Kit] Manufacturer'] = dict_2['catalogname']

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['[Kit] Manufacturer']},{dict_kit[str(i)]['[Kit] Part Number']},{dict_kit[str(i)]['[Kit] Type']},{dict_kit[str(i)]['[Kit] Type Code']}'''

    # = = = = = = = = = = = = = = =

    list_a = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
    info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
    dict_kit[str(i)]['[Kit] Info'] = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['[Kit] Part Code']}&pt={dict_kit[str(i)]['[Kit] Type Code']}&Lennon=1'

    # = = = = = = = = = = = = = = =

    list_input = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
    list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

    dict_src, j = {}, 0
    for dict_ in list_dict:
        list_src = dict_.get('Slots', [])
        for src in list_src:
            dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
            j += 1

    dict_kit[str(i)]['[Kit] Dict_Src'] = dict_src

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['[Kit] Manufacturer']};{dict_kit[str(i)]['[Kit] Part Number']};{dict_kit[str(i)]['[Kit] Type Code']};{dict_kit[str(i)]['[Kit] Part Code']}'''

dict_kit

{'0': {'[Kit] Quantity': 1,
  '[Kit] Type': 'Timing Chain',
  '[Kit] Type Code': '5724',
  '[Kit] Part Code': '8306448',
  '[Kit] Part Number': '715F',
  '[Kit] Manufacturer': 'MELLING',
  '[Kit] Url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724',
  '[Kit] Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
  '[Kit] Dict_Src': {'0': 'https://www.rockauto.com/info/583/715F-1-ANG.jpg'},
  '[Kit] JOIN_MPNTCPC': 'MELLING;715F;5724;8306448'},
 '1': {'[Kit] Quantity': 1,
  '[Kit] Type': 'Timing Chain Guide / Damper',
  '[Kit] Type Code': '5728',
  '[Kit] Part Code': '8307424',
  '[Kit] Part Number': 'BG5515',
  '[Kit] Manufacturer': 'MELLING',
  '[Kit] Url': 'https://www.rockauto.com/en/parts/MELLING,BG5515,Timing Chain Guide / Damper,5728',
  '[Kit] Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728',
  '[Kit] Dict_Src': {'0': 'https://www.rockauto.com/info/583/BG5515-1-SID.jpg'},
  '[Kit] JOIN_MPNTCPC': 'MELLING;BG5515;5728

In [8]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Group': crawler_series['Group'],
                          'Type': crawler_series['Type'],
                          'Manufacturer': crawler_series['Manufacturer'],
                          'Part Number': crawler_series['Part Number'],
                          'OEM': crawler_series['OEM'],
                          'Picture': crawler_series['Picture'],
                          'Url': crawler_series['Url'],
                          'Json_Src': crawler_series['Json_Src'],
                          'Info': crawler_series['Info'],
                          'Json_Note 1': crawler_series['Json_Note 1'],
                          'Json_Note 2': crawler_series['Json_Note 2'],
                          'Choose': crawler_series['Choose'],
                          'Json_Price': crawler_series['Json_Price'],
                          'Json_Specification': json.dumps(dict_specification),
                          'Json_Kit': json.dumps(dict_kit),
                          'Type Code': crawler_series['Type Code'],
                          'Part Code': crawler_series['Part Code'],
                          'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC']})

list_dict_correct

[{'No': 1,
  'Group': 'Engine',
  'Type': 'Timing Chain & Component Kit',
  'Manufacturer': 'MELLING',
  'Part Number': '30250S',
  'OEM': '',
  'Picture': '',
  'Url': 'https://www.rockauto.com/en/parts/melling,30250S,timing+chain+&+component+kit,5756',
  'Json_Src': '{"0": "https://www.rockauto.com/info/583/3-0250S-1-KIT.jpg", "1": "https://www.rockauto.com/info/583/BT7015-2-SID.jpg", "2": "https://www.rockauto.com/info/583/BG5515-1-SID.jpg", "3": "https://www.rockauto.com/info/583/715F-1-ANG.jpg", "4": "https://www.rockauto.com/info/583/S906A-1-FRO.jpg", "5": "https://www.rockauto.com/info/583/BT5514-1-ANG.jpg", "6": "https://www.rockauto.com/info/583/717F-1-ANG.jpg", "7": "https://www.rockauto.com/info/583/BT7016-1-TOP.jpg", "8": "https://www.rockauto.com/info/583/S899-1-FRO.jpg", "9": "https://www.rockauto.com/info/583/S905-1-FRO.jpg", "10": "https://www.rockauto.com/info/583/BG7019-2-SID.jpg"}',
  'Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756',
  'Json

In [9]:
crawler_status = True

crawler_status

True

In [10]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 OEM                                                              
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Json_Src        {"0": "https://www.rockauto.com/info/583/3-025...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                                                    {}
 Json_Note 2       {"0": "Category: Timing Chain & Component Kit"}
 Choose                                                           
 Json_Price                                       {"0": "$171.79"}
 Type Code                                                    

In [11]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_info.xlsx', index=False)

df_correct

,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Json_Specification,Json_Kit,Type Code,Part Code,JOIN_MPNTCPC
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}","{""0"": {""0"": ""Camshaft Sprocket Material"", ""1"":...","{""0"": {""[Kit] Quantity"": 1, ""[Kit] Type"": ""Tim...",5756,8304960,MELLING;30250S;5756;8304960


In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_info_error.xlsx', index=False)

df_error

,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC,Request_Url
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}",5756,8304960,MELLING;30250S;5756;8304960,https://www.rockauto.com/en/moreinfo.php?pk=83...
